# Logistic Regression: Predicting Affairs


## Dataset

The [Affair dataset](http://statsmodels.sourceforge.net/stable/datasets/generated/fair.html) was derived from a survey of women in 1974 by Redbook magazine, in which married women were asked about their participation in extramarital affairs. More information about the study is available in a [1978 paper](http://fairmodel.econ.yale.edu/rayfair/pdf/1978a200.pdf) from the Journal of Political Economy.

## Description of Variables

The dataset contains 6366 observations of 9 variables:

* `rate_marriage`: woman's rating of her marriage (1 = very poor, 5 = very good)
* `age`: woman's age
* `yrs_married`: number of years married
* `children`: number of children
* `religious`: woman's rating of how religious she is (1 = not religious, 4 = strongly religious)
* `educ`: level of education (9 = grade school, 12 = high school, 14 = some college, 16 = college graduate, 17 = some graduate school, 20 = advanced degree)
* `occupation`: woman's occupation (1 = student, 2 = farming/semi-skilled/unskilled, 3 = "white collar", 4 = teacher/nurse/writer/technician/skilled, 5 = managerial/business, 6 = professional with advanced degree)
* `occupation_husb`: husband's occupation (same coding as above)
* `affairs`: time spent in extra-marital affairs

## Problem Statement

This dataset can be treated as a classification dataset by creatign a new binary variable `affair` (did the women have at least one affair?). The problem is to predict the classification (have affair or not) for each women.

## Training the dragon!

In [1]:
# Import 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

### Load datasest


In [2]:
# load dataset
data = pd.read_csv("./data/affairs_dataset.csv")

# add "affair" column: 1 represents having affairs, 0 represents not
data['affair'] = (data.affairs > 0).astype(int)

FileNotFoundError: [Errno 2] File b'./data/affairs_dataset.csv' does not exist: b'./data/affairs_dataset.csv'

### Data Exploration

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.groupby('affair').mean()

We can see that on average, women who have affairs rate their marriages lower, which is to be expected. Let's take another look at the `rate_marriage` variable.

In [ ]:
data.groupby('rate_marriage').mean()

An increase in `age`, `yrs_married`, and `children` appears to correlate with a declining marriage rating.

## Data Visualization

In [ ]:
# show plots in the notebook
%matplotlib inline

Let's start with histograms of education and marriage rating.

In [ ]:
# histogram of education
data.educ.hist()
plt.title('Histogram of Education')
plt.xlabel('Education Level')
plt.ylabel('Frequency')

In [ ]:
# histogram of marriage rating
data.rate_marriage.hist()
plt.title('Histogram of Marriage Rating')
plt.xlabel('Marriage Rating')
plt.ylabel('Frequency')

Let's take a look at the distribution of marriage ratings for those having affairs versus those not having affairs.

In [ ]:
# barplot of marriage rating grouped by affair (True or False)
pd.crosstab(data.rate_marriage, data.affair.astype(bool)).plot(kind='bar')
plt.title('Marriage Rating Distribution by Affair Status')
plt.xlabel('Marriage Rating')
plt.ylabel('Frequency')

Let's use a stacked barplot to look at the percentage of women having affairs by number of years of marriage.

In [ ]:
affair_yrs_married = pd.crosstab(data.yrs_married, data.affair.astype(bool))
affair_yrs_married.div(affair_yrs_married.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Affair Percentage by Years Married')
plt.xlabel('Years Married')
plt.ylabel('Percentage')

### Preprocess data

In [ ]:
# # create dataframes with an intercept column and dummy variables for
# # occupation and occupation_husb
# from patsy import dmatrices
# Y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
#                   religious + educ + C(occupation) + C(occupation_husb)',
#                   data, return_type="dataframe")

# #del X["Intercept"]

# # fix column names of X
# X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
#                         'C(occupation)[T.3.0]':'occ_3',
#                         'C(occupation)[T.4.0]':'occ_4',
#                         'C(occupation)[T.5.0]':'occ_5',
#                         'C(occupation)[T.6.0]':'occ_6',
#                         'C(occupation_husb)[T.2.0]':'occ_husb_2',
#                         'C(occupation_husb)[T.3.0]':'occ_husb_3',
#                         'C(occupation_husb)[T.4.0]':'occ_husb_4',
#                         'C(occupation_husb)[T.5.0]':'occ_husb_5',
#                         'C(occupation_husb)[T.6.0]':'occ_husb_6'})

In [ ]:
X = data[["rate_marriage","age","yrs_married","children","religious", "educ", "occupation","occupation_husb"]]
Y = data["affair"]
print(X)

In [ ]:
# flatten Y into a 1-D array
Y = np.ravel(Y)

### Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(X, Y)

# check the accuracy on the training set
model.score(X.values, Y)

73% accuracy seems good, but what's the null error rate?

In [ ]:
# what percentage had affairs?
y.mean()

Only 32% of the women had affairs, which means that you could obtain 68% accuracy by always predicting "no". So we're doing better than the null error rate, but not by much.

Let's examine the coefficients to see what we learn.

In [ ]:
# examine the coefficients
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

### Model Evaluation Using a Validation Set


In [ ]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

We now need to predict class labels for the test set. We will also generate the class probabilities, just to take a look.

In [ ]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print(predicted)

In [ ]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print(probs)

Now let's generate some evaluation metrics.

In [ ]:
# generate evaluation metrics
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, probs[:, 1]))

The accuracy is 73%, which is the same as we experienced when training and predicting on the same data.

We can also see the confusion matrix and a classification report with other metrics.

In [ ]:
print(metrics.confusion_matrix(y_test, predicted))
print(metrics.classification_report(y_test, predicted))

### Predicting the Probability of an Affair

Just for fun, let's predict the probability of an affair for a random woman not present in the dataset. She's a 25-year-old teacher who graduated college, has been married for 3 years, has 1 child, rates herself as strongly religious, rates her marriage as fair, and her husband is a farmer.

Dataset attribute reference

* `rate_marriage`: woman's rating of her marriage (1 = very poor, 5 = very good)
* `age`: woman's age
* `yrs_married`: number of years married
* `children`: number of children
* `religious`: woman's rating of how religious she is (1 = not religious, 4 = strongly religious)
* `educ`: level of education (9 = grade school, 12 = high school, 14 = some college, 16 = college graduate, 17 = some graduate school, 20 = advanced degree)
* `occupation`: woman's occupation (1 = student, 2 = farming/semi-skilled/unskilled, 3 = "white collar", 4 = teacher/nurse/writer/technician/skilled, 5 = managerial/business, 6 = professional with advanced degree)
* `occupation_husb`: husband's occupation (same coding as above)
* `affairs`: time spent in extra-marital affairs

In [ ]:

X.columns

In [ ]:
model.predict_proba(np.array([[3, 25, 3, 1, 4, 16,4,2]]))

The predicted probability of an affair is 25%.

### AI is Love 